# Geospatial Analysis of Astronomical Data

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point
import folium
from scipy.stats import norm
from scipy.optimize import curve_fit
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, roc_curve
from sklearn.neural_network import MLPClassifier


## Load and Preprocess the Data

In [ ]:

# Load data
file_path = '/mnt/data/skyserver-modified.csv'
dtype_dict = {
    "class": "category",
    "u": "float64",      
    "g": "float64",
    "r": "float64",
    "i": "float64",
    "z": "float64",
    "redshift": "float64",
    "ra": "float64",
    "dec": "float64"
}
df = pd.read_csv(file_path, dtype=dtype_dict)
df.head()


## Convert to Geospatial Data

In [ ]:

# Convert to geospatial data
geometry = [Point(xy) for xy in zip(df['ra'], df['dec'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

# Display GeoDataFrame info
gdf.info()


## Create an Interactive Sky Map

In [ ]:

# Create a folium map centered at median RA/Dec
m = folium.Map(location=[df['dec'].median(), df['ra'].median()], zoom_start=5)

# Add points to the map
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row['dec'], row['ra']],
        radius=3,
        color='blue' if row['class'] == 'STAR' else 'green' if row['class'] == 'GALAXY' else 'red',
        fill=True,
        fill_opacity=0.5
    ).add_to(m)

# Save map to HTML file
m.save("/mnt/data/skyserver_map.html")
m


## Perform Geospatial Clustering (DBSCAN)

In [ ]:

# Geospatial Clustering using DBSCAN
coords = df[['ra', 'dec']]
scaler = StandardScaler()
coords_scaled = scaler.fit_transform(coords)
dbscan = DBSCAN(eps=0.3, min_samples=10)
df['cluster'] = dbscan.fit_predict(coords_scaled)

# Plot clustered data
plt.figure(figsize=(8,6))
sns.scatterplot(x=df['ra'], y=df['dec'], hue=df['cluster'], palette='viridis', s=10)
plt.xlabel("Right Ascension (degrees)")
plt.ylabel("Declination (degrees)")
plt.title("Spatial Clustering of Astronomical Objects")
plt.show()


## Save Clustered Data

In [ ]:

# Save clustered data
df.to_csv("/mnt/data/skyserver_clustered.csv", index=False)
